# Tutorials for training Models

In [1]:
import os
import tensorflow as tf
import glob
%load_ext autoreload
%autoreload 2
%matplotlib inline

## Preparing for Data

- Data preparing in CNN-based super-resolution research is always based on one assumption: Lr image is the `bicubic` downsampled version of Hr image. However we support many kind of degradations such as `gaussian` kernel, `bilinear` kernel, additive noise and so on. 

- We usually crop images into patches for convenience of training, but reconstruct the whole image when testing (when full convolution net). Based on the fact that the order of downsampling and cropping operation doesn't matter, I suggest to crop the image and save patches to `tfrecord` file first, then use `map` method to downsample each hr-patch.

In [2]:
train_dir = "./Image/set14" # Arbitrary
valid_dir = "./Image/set5"
AUTOTUNE = tf.data.experimental.AUTOTUNE
SCALE = 3

In [3]:
from src.write2tfrec import write_dst_tfrec, load_tfrecord

cache_dir = "./cache"
os.makedirs(cache_dir, exist_ok=True)

Saving patch into tfrecord file. IF saved, we can use directly.

In [6]:

if not os.path.isfile("./cache/set14_train_48x48.tfrec"):
    paths = list(glob.glob(os.path.join(train_dir, "*")))
    write_dst_tfrec(paths, 10, 48, "./cache/set14_train_48x48.tfrec")
    
if not os.path.isfile("./cache/set5_valid_48x48.tfrec"):
    paths = list(glob.glob(os.path.join(valid_dir, "*")))
    write_dst_tfrec(paths, 10, 48, "./cache/set5_valid_48x48.tfrec")



========================================WRITING TO TFRECORD=========================================


12it [00:00, 39.19it/s]


========================================WRITING TO TFRECORD=========================================


5it [00:00, 42.13it/s]


Input should be hr-patch, and output should be data pair (inputs, labels).
The degradation function used here is pre-defined in `preprocess.py` file, one can 
customize if needed.

In [4]:
from src.preprocess import degrade_image

def preprocess(hr):
    lr, hr = degrade_image(hr, SCALE, method=-1, restore_shape=False, kernel_sigma=0.5)
    return lr, hr


Load tfrecord file and map the preprocess function. 
`repeat()` makes the dataset repeat infinitely.

In [5]:
trdst = load_tfrecord(48, "./cache/set14_train_48x48.tfrec").map(preprocess).repeat()
valdst = load_tfrecord(48, "./cache/set5_valid_48x48.tfrec").map(preprocess)


## Train a pre-defined model

In [6]:
from src.model import EDSR_baseline

Here we train EDSR-baseline model for example. 

In [7]:
model = EDSR_baseline(scale=SCALE, model_name="EDSR_Baseline",
                      channel=3).create_model(load_weights=False,
                                              weights_path=None)


In [9]:
model.fit(trdst,
          valdst,
          nb_epochs=2,
          steps_per_epoch=20,
          batch_size=16,
          use_wn=False)

Training model : EDSR_Baseline_X3


W0623 15:55:35.973358 12892 training_utils.py:1353] Expected a shuffled dataset but input dataset `x` is not shuffled. Please invoke `shuffle()` on input dataset.
W0623 15:55:36.222787 12892 tf_logging.py:161] Model failed to serialize as JSON. Ignoring... 


Epoch 1/2
19/20 [===========================>..] - ETA: 1:56 - loss: 0.0978 - psnr_tf: 10.32 - ETA: 57s - loss: 0.0972 - psnr_tf: 10.4201 - ETA: 36s - loss: 0.1054 - psnr_tf: 10.131 - ETA: 25s - loss: 0.1045 - psnr_tf: 10.134 - ETA: 19s - loss: 0.0983 - psnr_tf: 10.423 - ETA: 15s - loss: 0.0924 - psnr_tf: 10.734 - ETA: 12s - loss: 0.0962 - psnr_tf: 10.604 - ETA: 8s - loss: 0.0898 - psnr_tf: 10.946 - ETA: 6s - loss: 0.0872 - psnr_tf: 11.07 - ETA: 5s - loss: 0.0865 - psnr_tf: 11.11 - ETA: 4s - loss: 0.0842 - psnr_tf: 11.23 - ETA: 3s - loss: 0.0814 - psnr_tf: 11.42 - ETA: 2s - loss: 0.0795 - psnr_tf: 11.54 - ETA: 2s - loss: 0.0787 - psnr_tf: 11.58 - ETA: 1s - loss: 0.0775 - psnr_tf: 11.63 - ETA: 1s - loss: 0.0757 - psnr_tf: 11.74 - ETA: 0s - loss: 0.0751 - psnr_tf: 11.77 - ETA: 0s - loss: 0.0741 - psnr_tf: 11.8186
Epoch 00001: saving model to ./weights/EDSR_Baseline_X3.h5
20/20 [==============================] - 8s 411ms/step - loss: 0.0725 - psnr_tf: 11.9326 - val_loss: 0.0542 - val_psnr

**Model's Info**

In [10]:
model.model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 6 1792        input_1[0][0]                    
__________________________________________________________________________________________________
res0 (_ResBlock)                (None, None, None, 6 73856       conv2d[0][0]                     
__________________________________________________________________________________________________
res1 (_ResBlock)                (None, None, None, 6 73856       res0[0][0]                       
______________________________________________________________________________________________